<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E5%9B%9B%E9%80%B1_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').sheet1

In [3]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [4]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [5]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


In [6]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=7d194697cdd0d5312e59014ca9421d87eb8549b9e80bdee8b3e7ddf432f869ac
  Stored in directory: /root/.cache/pip/wheels/8a/0a/37/f15b8568f5463f1427466f701e9d3ba514035eb703f885efee
Successfully built snownlp


In [7]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


In [8]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.276 seconds.
DEBUG:jieba:Loading model cost 1.276 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [9]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [10]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [11]:
df['Tokenized']

,Tokenized
0,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,..."
1,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現..."
2,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ..."
3,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,..."
4,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,..."
...,...
307,"[我, 是, 嘉工, 陳信錩, 希望, 可以, 學到, 東西]"
308,"[我, 是, 南港高中, 高廷翰, 對, 程式, 基礎, 有, 幫助]"
309,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."
310,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."


In [12]:
import re
import pandas as pd

# 定義角色判定的助教名單
assistants = ['pupupeter', 'Junxian0829', 'tedlu-tw']  # 根據先前提供的名單

# 定義角色判定規則的函數
def determine_role(row):
    if row['Account'] in assistants:
        return '助教'
    elif '助教' in row['Cleaned_Content'] or 'Collaborator' in row['Cleaned_Content']:
        return '助教'
    elif any(keyword in row['Cleaned_Content'] for keyword in ['老師', '數學老師']):
        return '老師'
    elif re.search(r'(高中|附中|高工|高商|工商)', row['Cleaned_Content']):
        return '學生'
    elif re.search(r'(大學|台師大)', row['Cleaned_Content']):
        return '大學生'
    else:
        return '未知'

# 定義學校提取函數
def extract_school(content):
    # 特例處理：如果提到特定學校，直接匹配
    if '數位實中' in content:
        return '數位實中'
    if '台師大' in content or '臺師大' in content or '師範大學' in content:
        return '臺師大'

    # 正則表達式匹配學校名稱
    school_keywords = r'(高中|附中|高工|高商|工商|大學|中學|國中|國小)'
    match = re.search(r'([\u4e00-\u9fa5]+?' + school_keywords + ')', content)
    if match:
        school = match.group(0)
        # 截取最後四個字作為學校名稱
        return school[-4:]
    return None

# 提取學校名稱
df['School'] = df['Cleaned_Content'].apply(lambda x: extract_school(str(x)))

# 應用函數到數據
df['Role'] = df.apply(determine_role, axis=1)
df['School'] = df['Cleaned_Content'].apply(lambda x: extract_school(str(x)))

In [35]:
df

,Account,Date,Cleaned_Content,Chinese_Content,Tokenized,School,Role
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...,學校姓名以及你為什麼來上這門課並留下你的第一個程式連結的我想要透過寫程式來實現我心中的想法我...,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,...",None,未知
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...,高中生們兒早哇我是這次的助教我叫家瑋現為台灣師範大學歷史系三年級的同學也是個愛大便的小孩子這...,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現...",臺師大,助教
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...,各位好我是鄭鈞賢是台師大機電系大三的從前我的夢想是成為一名頂尖的歌手可惜因為一些個人因素我唱...,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ...",臺師大,助教
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...,各位好和大家一樣是高中生就讀北市數位實中同時在臺大資工擔任研究助理雖然如此但我總是宣稱自己是...,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,...",數位實中,助教
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...,大家好我是胡程勛目前高二就讀彰化縣溪湖高中班在班上擔任的幹部是設備股長下面的程式是我在高一的...,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,...",溪湖高中,學生
...,...,...,...,...,...,...,...
307,xznxzn970312,"Sep 12, 2024",xznxzn970312 我是嘉工陳信錩 希望可以學到東西 https://colab.r...,我是嘉工陳信錩希望可以學到東西,"[我, 是, 嘉工, 陳信錩, 希望, 可以, 學到, 東西]",None,未知
308,Kaotinghan,"Sep 13, 2024",Kaotinghan 我是南港高中高廷翰 chatgpt對程式0基礎有幫助 https:/...,我是南港高中高廷翰對程式基礎有幫助,"[我, 是, 南港高中, 高廷翰, 對, 程式, 基礎, 有, 幫助]",南港高中,學生
309,fishpersonn,"Sep 28, 2024",fishpersonn 我是鳳梨大學傳播系的漁人 在網路上路過一起來學的！ 以下程式碼可以...,我是鳳梨大學傳播系的漁人在網路上路過一起來學的以下程式碼可以生成圖片喔是我在課上來的哈,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ...",鳳梨大學,大學生
310,pupupeter,"Sep 28, 2024",Collaborator pupupeter 我是鳳梨大學傳播系的漁人 在網路上路過一起來...,我是鳳梨大學傳播系的漁人在網路上路過一起來學的以下程式碼可以生成圖片喔是我在課上來的哈,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ...",鳳梨大學,助教


In [15]:
from collections import Counter

school_counts = Counter(df['School'])

In [16]:
school_counts

Counter({None: 39,
         '臺師大': 2,
         '數位實中': 2,
         '溪湖高中': 13,
         '中山附中': 25,
         '中山大學': 4,
         '南投高中': 2,
         '羅東高工': 14,
         '鼓山高中': 33,
         '仁武高中': 12,
         '關山工商': 2,
         '東海國中': 1,
         '中山高中': 9,
         '花蓮高工': 7,
         '楠梓高中': 17,
         '中港高中': 6,
         '清水高中': 31,
         '新社高中': 2,
         '林園高中': 15,
         '基隆高中': 30,
         '嘉義高工': 13,
         '南港高中': 19,
         '新竹高商': 10,
         '竹北高中': 2,
         '鳳梨大學': 2})

In [17]:
import plotly.express as px

# Sorting the school counts in descending order
sorted_school_counts = sorted(school_counts.items(), key=lambda x: x[1], reverse=True)
sorted_school_counts_dict = dict(sorted_school_counts)

# Creating a sorted DataFrame for the bar chart
sorted_school_df = pd.DataFrame(list(sorted_school_counts_dict.items()), columns=["School", "Count"])

# Plotting the sorted school distribution
fig_sorted = px.bar(sorted_school_df, x='School', y='Count',
                    title="School Distribution (Sorted by Count)",
                    labels={"School": "School", "Count": "Number of Mentions"},
                    text='Count')

fig_sorted.update_traces(texttemplate='%{text}', textposition='outside')
fig_sorted.update_layout(xaxis=dict(title="School"), yaxis=dict(title="Number of Mentions"))

fig_sorted.show()

In [18]:
# 計算每個 Account 的留言次數
account_message_count = df.groupby('Account').size().reset_index(name='Message_Count')

# 合併學校名稱到 account_message_count
account_school = df[['Account', 'School']].drop_duplicates()

# 合併留言次數與學校名稱
result = account_message_count.merge(account_school, on='Account')

In [19]:
result.describe()

,Message_Count
count,283.000000
mean,1.385159
std,1.565381
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,11.000000


In [20]:
result

,Account,Message_Count,School
0,40112,1,鼓山高中
1,40114,1,鼓山高中
2,210136,1,嘉義高工
3,218111,1,羅東高工
4,311257,1,清水高中
...,...,...,...
278,zhen910,1,花蓮高工
279,zhenjane,1,楠梓高中
280,zhenxinglin21024,1,基隆高中
281,zquonn,1,仁武高中


In [21]:
# Group the data by 'School' and aggregate the message count and accounts
grouped_data = result.groupby('School').agg({
    'Account': lambda accounts: ', '.join(map(str, accounts)),  # Convert integers to strings before joining
    'Message_Count': 'sum'
}).reset_index()

# Rename columns for clarity
grouped_data.rename(columns={
    'School': 'School_Name',
    'Account': 'Accounts',
    'Message_Count': 'Total_Messages'
}, inplace=True)

# Sort the grouped data by Total_Messages in descending order
sorted_data = grouped_data.sort_values(by='Total_Messages', ascending=False).reset_index(drop=True)
sorted_data

,School_Name,Accounts,Total_Messages
0,基隆高中,"21018Hank, An4567ny, Eeeeeeeemma, Kien0516, Le...",40
1,中山附中,"19-bryce, Anson594412, DingSianWu, I2amgay, Ia...",35
2,鼓山高中,"40112, 40114, 131001yt2019snipe, Andy980325, B...",33
3,清水高中,"311257, 311259, 311264, 311272, 10802-a, Allen...",31
4,楠梓高中,"971010ian, CaiBingyan123, Pei-0325, Rebecca-10...",27
5,羅東高工,"218111, 218234nick, JEFF-GMT, Jeff-1024, Jusus...",24
6,南港高中,"05yc, 1024Amon, 10909Jasmine, 10921Hennessy, 1...",19
7,溪湖高中,"Eric970308-123, Jiaxuan210357, Lin434255, Nily...",15
8,林園高中,"000000-Nora, Albert980206, Chuxin014, Irving-1...",15
9,中山大學,"Rna0925, kai081024, peculab",14


In [36]:
raw_accounts = list(sorted_data[sorted_data['School_Name'] == '基隆高中']['Accounts'])

In [37]:
raw_accounts

['21018Hank, An4567ny, Eeeeeeeemma, Kien0516, Lemon-Huan, Naomi726, RZKlove, RaY-smi1ec0w, Sherrycc-00, Timmmuoo, Xana1001, Yona825, bear97151106, chenliweileo, dilligaf01, dodo-6954, howard09166, jeremy970522, mattr-rrr, peculab, perry0407, t1na-5u, tim970520, tingchen601, txx1n, yongjun924, yuzn3348, zdujulie, zhenxinglin21024']

In [50]:
print(df[df['Account'] == 'QiuDaru']['Cleaned_Content'].values[0])

QiuDaru  大家好，我是東海國中邱達儒，因為想要讓AI幫我寫作業XD ( 開完笑的 ) (這感覺很帥所以我想學)，https://colab.research.google.com/drive/1S37huYuO6_YE4_QV9t6MBezCMePUD2pA?usp=sharing


In [44]:
id_list = raw_accounts[0].split(', ')
id_list[0]

'21018Hank'

In [51]:
print(df[df['Account'] == id_list[0]]['Cleaned_Content'].values[0])

21018Hank  我是基隆高中210班的游定衡，希望學好程式語言 https://colab.research.google.com/drive/1Ypk27I3NlMW9Lrm_JNuhc4SWAlpWW8S-?usp=sharing


In [24]:
raw_accounts = list(sorted_data[sorted_data['School_Name'] == '鳳梨大學']['Accounts'])

In [25]:
raw_accounts

['fishpersonn, pupupeter']

In [26]:
raw_accounts = list(sorted_data[sorted_data['School_Name'] == '基隆高中']['Accounts'])
raw_accounts

['21018Hank, An4567ny, Eeeeeeeemma, Kien0516, Lemon-Huan, Naomi726, RZKlove, RaY-smi1ec0w, Sherrycc-00, Timmmuoo, Xana1001, Yona825, bear97151106, chenliweileo, dilligaf01, dodo-6954, howard09166, jeremy970522, mattr-rrr, peculab, perry0407, t1na-5u, tim970520, tingchen601, txx1n, yongjun924, yuzn3348, zdujulie, zhenxinglin21024']

In [27]:
accounts_to_extract = set(raw_accounts[0].split(', '))

In [28]:
accounts_to_extract

{'21018Hank',
 'An4567ny',
 'Eeeeeeeemma',
 'Kien0516',
 'Lemon-Huan',
 'Naomi726',
 'RZKlove',
 'RaY-smi1ec0w',
 'Sherrycc-00',
 'Timmmuoo',
 'Xana1001',
 'Yona825',
 'bear97151106',
 'chenliweileo',
 'dilligaf01',
 'dodo-6954',
 'howard09166',
 'jeremy970522',
 'mattr-rrr',
 'peculab',
 'perry0407',
 't1na-5u',
 'tim970520',
 'tingchen601',
 'txx1n',
 'yongjun924',
 'yuzn3348',
 'zdujulie',
 'zhenxinglin21024'}

In [29]:
filtered_df = df[df['Account'].isin(accounts_to_extract)]

In [30]:
filtered_df

,Account,Date,Cleaned_Content,Chinese_Content,Tokenized,School,Role
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...,學校姓名以及你為什麼來上這門課並留下你的第一個程式連結的我想要透過寫程式來實現我心中的想法我...,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,...",None,未知
13,peculab,"Sep 2, 2024",Owner Author peculab 大家好，我是王俊剴，目前高一，讀高雄市中山大學附...,大家好我是王俊剴目前高一讀高雄市中山大學附中高一丙班對程式感興趣未來也想往這方面發展,"[大家, 好, 我, 是, 王俊剴, 目前, 高一, 讀, 高雄市, 中山, 大學, 附中,...",中山大學,學生
103,peculab,"Sep 3, 2024",Owner Author peculab 大家好，我是羅東高工電機科的李博堯，希望能對AI...,大家好我是羅東高工電機科的李博堯希望能對有更多的了解所以選擇這門課,"[大家, 好, 我, 是羅東, 高工, 電機科, 的, 李博堯, 希望, 能, 對, 有, ...",羅東高工,學生
138,peculab,"Sep 5, 2024",Owner Author peculab I'm Pecu. My code is htt...,,[],None,未知
139,peculab,"Sep 5, 2024","Owner Author peculab 大家好，我是楊森宇,就讀中山附中高一丙班 想玩玩...",大家好我是楊森宇就讀中山附中高一丙班想玩玩看程式設計,"[大家, 好, 我, 是, 楊森宇, 就, 讀, 中山, 附中, 高一, 丙班想, 玩玩, ...",中山附中,學生
140,peculab,"Sep 5, 2024",Owner Author peculab 我是楠梓高中101黃軍燁 https://col...,我是楠梓高中黃軍燁,"[我, 是, 楠梓, 高中, 黃, 軍燁]",楠梓高中,學生
141,peculab,"Sep 5, 2024",Owner Author peculab 我是明天才要上課的陪同老師~ 今天先預習一下!!...,我是明天才要上課的陪同老師今天先預習一下本身對程式完全基礎的我從網頁到軟體對我來說都是新鮮的...,"[我, 是, 明天, 才, 要, 上, 課, 的, 陪同, 老師, 今天, 先, 預習, 一...",None,老師
165,perry0407,"Sep 5, 2024",perry0407 老師好，我是基隆高中210班的陳品丞，我希望能藉由學習程式感善生活環境...,老師好我是基隆高中班的陳品丞我希望能藉由學習程式感善生活環境,"[老師, 好, 我, 是, 基隆, 高中班, 的, 陳, 品丞, 我, 希望, 能, 藉由,...",基隆高中,老師
178,dilligaf01,"Sep 5, 2024","dilligaf01 大家好我是基隆高中210石兆馨,選這門課是因為覺得程式設計對未來有幫...",大家好我是基隆高中石兆馨選這門課是因為覺得程式設計對未來有幫助,"[大家, 好, 我, 是, 基隆, 高中, 石兆馨, 選, 這門課, 是, 因為, 覺得, ...",基隆高中,學生
179,Lemon-Huan,"Sep 5, 2024",Lemon-Huan 我是國立基隆高中210班的陳禹瑍，希望能夠藉此學習到更多有關AI的知...,我是國立基隆高中班的陳禹瑍希望能夠藉此學習到更多有關的知識並能夠應用在以後生活中的各種方面,"[我, 是, 國立, 基隆, 高中班, 的, 陳, 禹, 瑍, 希望, 能夠, 藉此, 學習...",基隆高中,學生


In [32]:
from google.colab import userdata
apikey = userdata.get('GEMINI_API_KEY')

In [34]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.5-flash")

prompt = f"用中文分析社交媒體上對特定事件的風向：{filtered_df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這批社交媒體數據記錄了某門程式設計或AI相關課程的學員（主要是學生，也有一位陪同老師）的自我介紹和學習動機。這些貼文集中在2024年8月底至9月初，是一個特定課程開展時的互動。

以下是針對這些社交媒體上對該特定事件的風向分析：

### 一、 事件概述

該事件的核心是**一門程式設計/AI課程的學員入學/開課前的自我介紹與學習動機分享**。學員們在社交媒體平台（可能是一個課程交流群或論壇）上發布信息，內容通常包含姓名、學校、班級，以及他們選擇這門課的原因和對學習的期望，並附上他們的第一個程式連結，表明這是一個實踐性較強的課程。

### 二、 參與群體分析

1.  **主體：** 絕大多數參與者為**高中學生**。他們以簡短明瞭的方式進行自我介紹，並表達學習意願。
2.  **學校分佈：** 觀察到一個非常顯著的特點是，**來自「基隆高中」的學生佔據了壓倒性的比例**，幾乎形成了集體報名或參加的現象。這可能暗示該課程與基隆高中有深度合作、是校內推廣活動的一部分，或者在該校學生群體中引發了極高的興趣。此外，也有少量學生來自「中山大學附中」、「羅東高工」和「楠梓高中」。
3.  **老師參與：** 有一位明確標示為「陪同老師」的用戶也參與了介紹，表示「對程式完全0基礎」，並希望「好好地陪同學生玩下去」。這展現了老師積極參與、與學生共同學習的開放態度，也暗示課程內容可能設計得對初學者友好。
4.  **活動發起者/講師：** 帳號`peculab`（Pecu）是本次活動的發起者或主要教學人員，其發布的內容包括介紹性示範和目標設定（如「五百五十萬台幣年薪」的目標程式碼），旨在激勵學員。

### 三、 風向/主要情緒和動機

整體而言，社交媒體上關於此事件的風向呈現出**高度的學習動機、積極的參與度以及對程式設計和人工智能領域的濃厚興趣**。

1.  **強烈的學習熱情與好奇心：**
    *   「對程式感興趣」、「想玩玩看程式設計」、「希望能對AI有更多的了解」等表達頻繁出現，顯示學生們對新知識、新技能充滿好奇和探索欲。
    *   「雖然看起來有點難，但我會認真學習的！」、「希望學好程式語言」等語句，體現了學員們面對挑戰的決心和積極的學習態度。

2.  **對AI的高度關注：**
    *   許多學生明確表示希望「學習到更多有關AI的知識」、「了解